#Converting the original metadata on the Books sample for Amazon reviews into a smaller DataFrame

The original metadata was a large 3.8GB file with more information than necessary - for the scope of our project - so we loaded into this notebook to change its format to a DataFrame and keep only the book titles, price, and ASIN codes we need to add to our existing reviews data.

In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#The original json file is in lines format - need the additional argument lines=True
book_titles = pd.read_json('/content/drive/MyDrive/BU_MSBA/BA820 - Unsupervised ML/meta_Books.json', lines=True)

In [4]:
book_titles.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')

In [13]:
#Check book categoory - It includes genres but in a list format

#Some values are missing and inputed as empty lists, change values to np.nan
book_titles['category'] = book_titles['category'].apply(lambda x: np.nan if not x else x)

#It seems like all items start with Books - which does not help much
#Additionally, other book state identifiers like New and Used are includes
#Remove those first
book_titles['category'].head(10)

0                                                  NaN
1    [Books, New, Used & Rental Textbooks, Medicine...
2                   [Books, Arts & Photography, Music]
3                   [Books, Arts & Photography, Music]
4                                                  NaN
5    [Books, New, Used & Rental Textbooks, Medicine...
6                     [Books, Medical Books, Medicine]
7    [Books, New, Used & Rental Textbooks, Business...
8    [Books, Education & Teaching, Schools & Teaching]
9                                                  NaN
Name: category, dtype: object

In [28]:
book_titles['category'].head(20)

0                                                   NaN
1     [Books, New, Used & Rental Textbooks, Medicine...
2                    [Books, Arts & Photography, Music]
3                    [Books, Arts & Photography, Music]
4                                                   NaN
5     [Books, New, Used & Rental Textbooks, Medicine...
6                      [Books, Medical Books, Medicine]
7     [Books, New, Used & Rental Textbooks, Business...
8     [Books, Education & Teaching, Schools & Teaching]
9                                                   NaN
10                                                  NaN
11                                                  NaN
12                [Books, New, Used & Rental Textbooks]
13                                                  NaN
14                [Books, Humor & Entertainment, Radio]
15    [Books, Engineering & Transportation, Engineer...
16        [Books, Literature & Fiction, Dramas & Plays]
17    [Books, New, Used & Rental Textbooks, Scie

In [38]:
#Removing 'Books', 'New', 'Used' from categories

#Create function to remove item from list
def remove_item_from_list(lst, items_to_remove):
  """Takes a list (target), and remove another list of items from the target
   list
  """

  #First check of item is list (and not NaN)
  if isinstance(lst, list):
        return [item for item in lst if item not in items_to_remove]

  #If it as not a list we return the value (NaN)
  return lst

items_to_remove = ['Books',  'New, Used & Rental Textbooks']

# Remove 'Books', 'New', and 'Used & Rental Textbooks'
book_titles['category'] = book_titles['category'].apply(lambda x:\
                                                         remove_item_from_list(x, items_to_remove))



In [75]:
#Checking most common number of categories for books - its 2
book_titles['category'].apply(lambda x: len(x) if isinstance(x, list) else np.nan).mode()

0    2.0
Name: category, dtype: float64

In [77]:
#Create new columns - main_genre, secondary_genre

#Create function to get main_genre and secondary_genre
def extract_genres(lst):
  """Receives a list, checks if value is a list. If it is, the first item will be
  the main_genre, and if the list has more than one item the second item becomes
  the secondary_genre. If the value passed is not a list both main_genre and
  secondary_genre return null
  """

  if isinstance(lst, list):
      main_genre = lst[0] if len(lst) > 0 else np.nan
      secondary_genre = lst[1] if len(lst) > 1 else np.nan
  else:
      main_genre, secondary_genre = np.nan, np.nan
  return [main_genre, secondary_genre]


#Apply function
book_titles[['main_genre', 'secondary_genre']] = book_titles['category'].apply(extract_genres).apply(pd.Series)


In [79]:
#Select the title, asin and price columns
books_meta = book_titles[['title', 'asin', 'price', 'main_genre', 'secondary_genre']]

In [80]:
#Missing values in price are marked as '', replace with NaN
books_meta.loc[:,'price'] = books_meta['price'].replace('', np.nan)

<ipython-input-80-739d00752f8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_meta.loc[:,'price'] = books_meta['price'].replace('', np.nan)


In [81]:
#Size of file was reduced from 3.8GB to 241MB
books_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2934949 entries, 0 to 2934948
Data columns (total 5 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   title            object
 1   asin             object
 2   price            object
 3   main_genre       object
 4   secondary_genre  object
dtypes: object(5)
memory usage: 112.0+ MB


In [82]:
#Save new dataframe
from google.colab import files

#Convert into csv
books_meta.to_csv('books_meta_short_with_genre.csv')

#Download new csv file
files.download('books_meta_short_with_genre.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>